In this project, I'm working with a CIA database of world facts to complete an analysis using SQL. 

Developed by [Ahmed Kayal](https://www.linkedin.com/in/ahmedkayal/). 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

#### View of the tables currently present in the CIA database

In [2]:
%%sql
SELECT * FROM sqlite_master

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT * 
FROM facts 
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [4]:
%%sql
SELECT COUNT(DISTINCT(name)) AS country_count
FROM facts

Done.


country_count
261


The above count reflects the number of unique countries in this database. There are a varying number of globally recognized countries and this count is in line with a few of them. 

Quick look at the database's maximum and minimum values for population and population growth. 

In [5]:
%%sql
SELECT MIN(population) AS minimum_population, MAX(population) AS maximum_population, MIN(population_growth)  AS minimum_population_growth, MAX(population_growth) AS maximum_population_growth
FROM facts 


Done.


minimum_population,maximum_population,minimum_population_growth,maximum_population_growth
0,7256490011,0.0,4.02


Surprised to see those values for minimum population, maximum population, and minimum population growth. I'll double check that anlysis:

In [6]:
%%sql
SELECT name, area, population, population_growth 
FROM facts
WHERE population = (SELECT MIN(population)
                   FROM facts) 
    OR population = (SELECT MAX(population)
                   FROM facts)
    OR population_growth = (SELECT MIN(population_growth)
                   FROM facts)

Done.


name,area,population,population_growth
Holy See (Vatican City),0,842,0.0
Cocos (Keeling) Islands,14,596,0.0
Greenland,2166086,57733,0.0
Pitcairn Islands,47,48,0.0
Antarctica,None,0,None
World,None,7256490011,1.08


The above results reflects the names of the countries that are equal to the database's minimum population, maximum population, and the minimum population growth. Given these results, I'll reexecute my initial query while filtering out for these specified results. 

In [7]:
%%sql
SELECT MIN(population) AS minimum_population, MAX(population) AS maximum_population, MIN(population_growth)  AS minimum_population_growth, MAX(population_growth) AS maximum_population_growth
FROM facts 
WHERE name NOT IN (SELECT name 
                   FROM facts 
                  WHERE population = (SELECT MIN(population)
                   FROM facts) 
                    OR population = (SELECT MAX(population)
                   FROM facts)
                    OR population_growth = (SELECT MIN(population_growth)
                   FROM facts))



Done.


minimum_population,maximum_population,minimum_population_growth,maximum_population_growth
1190,1367485388,0.01,4.02


Executing average calculations for population and area columns

In [8]:
%%sql
SELECT AVG(population), AVG(area)
FROM facts
WHERE name NOT IN (SELECT name 
                   FROM facts 
                  WHERE population = (SELECT MIN(population)
                   FROM facts) 
                    OR population = (SELECT MAX(population)
                   FROM facts)
                    OR population_growth = (SELECT MIN(population_growth)
                   FROM facts))


Done.


AVG(population),AVG(area)
32925522.98305085,555314.8816326531


The results of these calculations are definitely being impacted by the outlier population sizes of the countries in this database. I'll take a quick look at the countries who have above average population and area sizes.

In [9]:
%%sql
SELECT name, area, population, population_growth, birth_rate, death_rate
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts)
    AND area > (SELECT AVG(area)
               FROM facts)
ORDER BY population DESC
LIMIT 10

Done.


name,area,population,population_growth,birth_rate,death_rate
China,9596960,1367485388,0.45,12.49,7.53
India,3287263,1251695584,1.22,19.55,7.32
European Union,4324782,513949445,0.25,10.2,10.2
United States,9826675,321368864,0.78,12.49,8.15
Indonesia,1904569,255993674,0.92,16.72,6.37
Brazil,8515770,204259812,0.77,14.46,6.58
Pakistan,796095,199085847,1.46,22.58,6.49
Nigeria,923768,181562056,2.45,37.64,12.9
Russia,17098242,142423773,0.04,11.6,13.69
Mexico,1964375,121736809,1.18,18.78,5.26


For the final part of my analysis, I'd like to see how the results from the above table (the countries with the above average population and area sizes) compares to the countries with above average death rates. 

To do that, I'll first find the countries with above average death rates.

In [10]:
%%sql
SELECT name, population, death_rate
FROM facts
WHERE death_rate > (SELECT AVG(death_rate)
                   FROM facts)
LIMIT 10

Done.


name,population,death_rate
Afghanistan,32564342,13.89
Angola,19625353,11.49
Armenia,3056382,9.34
Austria,8665550,9.42
Barbados,290604,8.44
Belarus,9589689,13.36
Belgium,11323973,9.63
Benin,10448647,8.21
Bosnia and Herzegovina,3867055,9.75
Botswana,2182719,13.39


Finding these above average countries, I'll move to join them with my previous results.

In [11]:
%%sql
SELECT A.*
FROM (SELECT name, area, population, population_growth, birth_rate, death_rate
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts)
    AND area > (SELECT AVG(area)
               FROM facts)
ORDER BY population DESC) AS A INNER JOIN (SELECT name, population, death_rate
FROM facts
WHERE death_rate > (SELECT AVG(death_rate)
                   FROM facts)) AS B
WHERE A.name = B.name
LIMIT 10

Done.


name,area,population,population_growth,birth_rate,death_rate
European Union,4324782,513949445,0.25,10.2,10.2
United States,9826675,321368864,0.78,12.49,8.15
Nigeria,923768,181562056,2.45,37.64,12.9
Russia,17098242,142423773,0.04,11.6,13.69
Ethiopia,1104300,99465819,2.89,37.27,8.19
"Congo, Democratic Republic of the",2344858,79375136,2.45,34.88,10.07
France,643801,66553766,0.43,12.38,9.16


Interesting to see how the countries with above average populations and above average death rates are actually some of the most developed countries in the world. 